# Interactive Brain Visualization

This is an interactive tool for visualizing time series data from an ECoG experiment. You can do the following:
* Select electrodes with the menu list or mouse selection tool (use ctrl-L or ctrl-C and keep ctrl pressed, then use the mouse)
* Change the size and color of selected and unselected electrodes

In [72]:
import neuropythy as ny
import ipyvolume as ipv
from matplotlib import cm
import numpy as np
import scipy.io
import os
import wget
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
from IPython.display import clear_output

# for unnzipping and loading data
from zipfile import ZipFile

# for dropdown menu
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, GridspecLayout, Layout

# For loading data files
import h5py

# from bqplot import pyplot as plt
from matplotlib import pyplot as plt

from bqplot import interacts
import warnings
warnings.filterwarnings("ignore")

In [73]:
## FETCH DATA

# Where is the data located?
local_path = '../shablona/data'
# local_path = '/Applications/freesurfer_v5/subjects/'
subj_path = local_path+'/test_subj'

# If the data is not there, download and extract it
if not os.path.isdir(os.path.join(local_path,'test_subj')):
    # download example data with wget
    print('Downloading mesh and electrode data with wget')
    myfile = wget.download('https://zenodo.org/record/996814/files/Hamilton_et_al_img_pipe_test_subj.zip?download=1', local_path)
    # extract files
    with ZipFile(myfile, 'r') as zip_ref:
        #print(zip_ref.printdir())
        # Technically we only need the surf, mri, and label directories
        #zip_ref.extract('test_subj/surf/', local_path)
        #zip_ref.extract('test_subj/mri/', local_path)
        zip_ref.extractall(local_path)

# Get the functional data
data_dict = dict()
with h5py.File('%s/test_subj_CV.hf5'%(local_path),'r') as f:
    stimulus_types = f['ecog'].keys()
    times = f['times'][:].ravel()
    #print(stimulus_types)
    for s in stimulus_types:
        data_dict[s] = f['ecog'][s][:]
        
# Get the electrode coordinates
if os.path.exists(subj_path):
    sub = ny.freesurfer_subject(subj_path)
    elecs = scipy.io.loadmat(os.path.join(subj_path,'elecs/TDT_elecs_all.mat'))
else:
    print('file not found?')
    sub = ny.freesurfer_subject('test_subj')
    elecs=scipy.io.loadmat('/Applications/freesurfer_v5/subjects/test_subj/elecs/TDT_elecs_all.mat')


In [74]:
def plot_activity(data_dict, times, condition, elec):
    '''
    Plot the activity for the selected electrode(s)
    Inputs:
    
        data_dict [dict] : 
        times [array-like] : 
        condition [str] : condition to plot (from data_dict.keys())
        elec [int] : selected electrode 
        
    '''
    if len(elec) > 1:
        data = data_dict[condition][elec,:].mean(0)
        data_std = data_dict[condition][elec,:].std(0)/np.sqrt(len(elec))
    else:
        data = data_dict[condition][elec,:]
        data_std = np.zeros((data.shape))
    fig = plt.figure()
    plt.plot(times, data)
    plt.fill_between(times, data-data_std, data+data_std, alpha=0.5)
    plt.axvline(color='k', linestyle='--', linewidth=1)
    plt.axhline(color='k', linestyle='--', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Z-scored HG')

#fig_out = widgets.Output(layout=Layout(height='300px', width = '300px', border='solid'))

#update_plot = interact(plot_activity, data_dict=data_dict, times=times, condition='CV_listen', elec=[0,1])

In [112]:
anatomical_areas = [area[0] for area in np.unique(elecs['anatomy'][:,3])]

def select_parameters(#selected_elec = elecs['eleclabels'][:,0], 
                      selected_stimulus = [k for k in data_dict.keys()],
                      #selected_time = (0, 10, 10000)
                        ):
#     elecs_scatter.selected = np.where(elecs['eleclabels'][:,0] == selected_elec)
    plot_activity(data_dict, times, condition = selected_stimulus, elec = elecs_scatter.selected)
    return

    
#elecs_scatter.selected

w = interactive(select_parameters)

elec_select = widgets.SelectMultiple(
   #options= {a[0]:i for i, a in enumerate(list(elecs['eleclabels'][:,0]))},
   options = {a[0][0]+'-'+a[1][0]: i for i, a in enumerate(zip(elecs['anatomy'][:,0], elecs['anatomy'][:,3]))},
   value= [1],
   #rows=10,
   description='Electrode(s)',
   disabled=False
)

output_elec_select = widgets.Output()

def on_value_change(change):
    with output_elec_select:
        elecs_scatter.selected = [i for i in change['new']]
        #plot_activity(data_dict, times, w.children[0].value, elec = elecs_scatter.selected)
        
    #with fig_out:
        #print(w.children[0].value)
    #    clear_output()
    #    plot_activity(data_dict, times, w.children[0].value, elec = elecs_scatter.selected)
    return

elec_select.observe(on_value_change, names='value')

In [113]:
# Make the brain white but slightly transparent (alpha=0.5)
clr = np.ones((sub.rh.pial_surface.vertex_count,4))
clr[:,3] = 0.5

# Initialize ipyvolume figure
ipv.figure()
ipf = ny.cortex_plot(sub.rh.pial_surface, color=clr, alpha=0.5, underlay=None, mesh_alpha = 0.5)

# Get the lasso selector
ipv.selector_default()

# Plot the electrodes as spheres
elecs_scatter = ipv.pylab.scatter(elecs['elecmatrix'][:,0],elecs['elecmatrix'][:,1],elecs['elecmatrix'][:,2], 
                            marker='sphere', selected=[0,1,2,3,4], color_selected='cornflowerblue', 
                            color='coral', size_selected = 3.0)

def on_lasso_choice(*_):
    elec_select.value =  tuple(elecs_scatter.selected[0]);
    w.children[0].value = w.children[0].value
#     line.set_ydata(np.zeros(data_dict['CV_listen'].shape[1],))
#     plt.show()

    
elecs_scatter.observe(on_lasso_choice,'selected')

style = {'description_width': '100px'}

size = FloatSlider(min=0, max=10, step=0.1, description='Unselected size', style=style)
size_selected = FloatSlider(min=0, max=10, step=0.1, description='Selected size', style=style)
color = ColorPicker(concise=True, description='Unselected', style=style)
color_selected = ColorPicker(concise=True, description='Selected', style=style)
jslink((elecs_scatter, 'size'), (size, 'value'))
jslink((elecs_scatter, 'size_selected'), (size_selected, 'value'))
jslink((elecs_scatter, 'color'), (color, 'value'))
jslink((elecs_scatter, 'color_selected'), (color_selected, 'value'))
# jslink((elecs_scatter, 'selected'),(elec_select, 'index'))
#VBox([ipv.gcc(), size, size_selected, color, color_selected])



grid = GridspecLayout(8, 3)
grid[:, 0] = ipf
grid[0, 1] = elec_select
grid[1:, 1] = w
#grid[2:,1] = fig_out

grid
# ipv.show()


Output()

GridspecLayout(children=(Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), q…